<a href="https://www.kaggle.com/code/jatinnk121/titanic-comp?scriptVersionId=143664990" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
org_train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
org_test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
org_train_data.head()

In [ ]:
org_train_data.describe()
org_test_data.describe()

In [ ]:
miss_vals = org_train_data.isnull()
print(miss_vals.sum())
miss_vals = org_test_data.isnull()
print(miss_vals.sum())
filled_train_data = org_train_data[['Pclass' , 'Sex'  , 'SibSp', 'Parch'  , 'Embarked', 'Age', 'Fare', 'Survived']].copy()
filled_test_data = org_test_data[['Pclass' , 'Sex'  , 'SibSp', 'Parch'  , 'Embarked', 'Age','Fare' ]].copy()
filled_train_data.head()

In [ ]:
mode_embarked = org_train_data['Embarked'].mode()[0]
filled_train_data['Embarked'].fillna(mode_embarked, inplace = True)
filled_train_data['Age'].fillna(org_train_data['Age'].mean(), inplace = True)
filled_test_data['Fare'].fillna(org_test_data['Fare'].mean(), inplace = True)
filled_test_data['Age'].fillna(org_test_data['Age'].mean(), inplace = True)
mis_vals = filled_train_data.isnull()
print(mis_vals.sum())
mis_vals = filled_test_data.isnull()
print(mis_vals.sum())

In [ ]:
filled_train_data['Fare'].hist(bins =20)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
numerical_scaler = MinMaxScaler()
encoded_train_data = filled_train_data.copy()
encoded_test_data = filled_test_data.copy()
encoded_train_data['Fare'] = numerical_scaler.fit_transform(filled_train_data['Fare'].values.reshape(-1, 1))
encoded_train_data['Age'] = numerical_scaler.fit_transform(filled_train_data['Age'].values.reshape(-1, 1))
encoded_test_data['Fare'] = numerical_scaler.fit_transform(filled_test_data['Fare'].values.reshape(-1, 1))
encoded_test_data['Age'] = numerical_scaler.fit_transform(filled_test_data['Age'].values.reshape(-1, 1))


In [ ]:
encoded_train_data.head()

In [ ]:
i=1
while i<5 :
    encoded_train_data.iloc[: , i] = LabelEncoder().fit_transform(filled_train_data.iloc[: , i])
    encoded_test_data.iloc[: , i] = LabelEncoder().fit_transform(filled_test_data.iloc[: , i])
    i+=1


In [ ]:
encoded_test_data.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test , Y_train , Y_test = train_test_split(encoded_train_data.iloc[: ,0:7] , encoded_train_data.iloc[:,7] , test_size = 0.2 , random_state =2)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model.fit(X_train , Y_train)

In [ ]:
from sklearn.metrics import accuracy_score
#Accuracy on training data
X_train_predicted = log_model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_predicted,Y_train)
# Accuracy on test data
X_test_predicted = log_model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_predicted,Y_test)
print('Accuracy on Training Data: ',round((training_data_accuracy*100) ,2) , "%")
print('Accuracy on Test Data: ',round((test_data_accuracy*100) ,2) , "%")

In [ ]:
import tensorflow as tf

# Define the input layer
input_layer = tf.keras.layers.Input(shape=(7,))

# Define the hidden layers with sigmoid activation
hidden_layer1 = tf.keras.layers.Dense(32, activation='relu')(input_layer)
hidden_layer2 = tf.keras.layers.Dense(64, activation='relu')(hidden_layer1)
hidden_layer3 = tf.keras.layers.Dense(32, activation='relu')(hidden_layer2)

# Define the output layer with one neuron and sigmoid activation
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_layer3)

# Create the neural network model
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

# Compile the model (you can choose your own loss and optimizer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
X_train = X_train.astype(np.float32)
Y_train = Y_train.astype(np.float32)
model.fit(X_train, Y_train, epochs=35, batch_size=32,validation_split=0.2)


In [ ]:
X_train.shape

In [ ]:
X_test = X_test.astype(np.float32)
Y_test = Y_test.astype(np.float32)
Y_pred = model.predict(X_test)
threshold = 0.5  # Threshold for binarization
test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

In [ ]:
print(len(Y_test))

In [ ]:
predictions = model.predict(encoded_test_data.loc[:,:].astype(np.float32))
predictions_2 = log_model.predict(encoded_test_data.loc[:,:].astype(np.float32))

In [ ]:
predictions_arr_2 = predictions_2.ravel()
final_predictions_2 = (predictions_2 > 0.5).astype(int)
flattened_array_2 = np.array(final_predictions_2).flatten()

In [ ]:
predictions_arr = predictions.ravel()
final_predictions = (predictions > 0.5).astype(int)
flattened_array = np.array(final_predictions).flatten()

In [ ]:
flattened_array_2


In [ ]:
output = pd.DataFrame({'PassengerId': org_test_data.PassengerId, 'Survived': flattened_array})

In [ ]:
output_2 = pd.DataFrame({'PassengerId': org_test_data.PassengerId, 'Survived': flattened_array})

In [ ]:
output.head()

In [ ]:
output_2.head()

In [ ]:
output.to_csv('submission.csv', index=False)

In [ ]:
output_2.to_csv('submission_2.csv', index=False)